In [1]:
import pybamm

In [3]:
model = pybamm.lithium_ion.SPMe()
param = pybamm.ParameterValues("Chen2020")
fast_solver = pybamm.CasadiSolver(atol=1e-3, rtol=1e-3, mode="fast")    

#”fast with events”: perform direct integration of the whole timespan, then go back and check where events were crossed. Experimental only. 
# 첫 번째 스텝을 위한 초기 솔루션 생성
# experiment = pybamm.Experiment(
#     [
#         "Discharge at 1C until 2.3V",
#         "Rest for 3 hours"
#     ]
# )
t_eval = [60,120]
fast_sim = pybamm.Simulation(model, parameter_values=param,solver=fast_solver)#, experiment=experiment

initial_solution = fast_sim.solve(t_eval)



In [10]:
import pandas as pd  # needed to read the csv data file
model = pybamm.lithium_ion.DFN()
parameter_values = pybamm.ParameterValues("Chen2020")

parameter_values["Current function [A]"] = 10
parameter_values["Open-circuit voltage at 100% SOC [V]"] = 3.4
parameter_values["Open-circuit voltage at 0% SOC [V]"] = 3.0


sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve([0, 3600], initial_soc=1)
sim.plot()


At t = 233.183 and h = 1.28053e-16, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 233.183 and h = 3.06498e-14, the corrector convergence failed repeatedly or with |h| = hmin.


interactive(children=(FloatSlider(value=0.0, description='t', max=136.5079474211171, step=1.3650794742111712),…

In [12]:
# Import drive cycle from file
drive_cycle = pd.read_csv(
    "US06.csv", comment="#", header=None
).to_numpy()

# Create interpolant
current_interpolant = pybamm.Interpolant(drive_cycle[:, 0], drive_cycle[:, 1], pybamm.t)

# Set drive cycle
parameter_values["Current function [A]"] = current_interpolant

In [14]:
drive_cycle

array([[0.0000e+00, 1.2859e-02],
       [1.0000e+00, 1.2859e-02],
       [2.0000e+00, 1.2859e-02],
       ...,
       [5.9800e+02, 1.2859e-02],
       [5.9900e+02, 1.2859e-02],
       [6.0000e+02, 1.2859e-02]])

In [13]:
model = pybamm.lithium_ion.SPMe()
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve()
sim.plot(["Current [A]", "Voltage [V]"])

interactive(children=(FloatSlider(value=0.0, description='t', max=600.0, step=6.0), Output()), _dom_classes=('…

In [9]:
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve([0, 3600], initial_soc=1)
sim.plot()

At t = 233.183 and h = 1.28053e-16, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 233.183 and h = 3.06498e-14, the corrector convergence failed repeatedly or with |h| = hmin.


interactive(children=(FloatSlider(value=0.0, description='t', max=136.5079474211171, step=1.3650794742111712),…

In [16]:
import numpy as np
t = np.linspace(0, 1, 60)
sin_t = 0.5 * np.sin(2 * np.pi * t)
drive_cycle_power = np.column_stack([t, sin_t])
experiment = pybamm.Experiment([pybamm.step.power(drive_cycle_power)])
sim = pybamm.Simulation(model, experiment=experiment)
sim.solve()
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

# CH5

In [ ]:
import pybamm
import numpy as np

experiment = pybamm.Experiment(
    [
        (
            "Discharge at C/10 for 10 hours or until 3.3 V",
            "Rest for 1hour",
            "Charge at 1 A until 4.1 V",
            "Hold at 4.1 V untile 50 mA"
            "Rest"
        )    
    ]
)

In [19]:
#%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed
import pybamm
import numpy as np
import matplotlib.pyplot as plt

full_model = pybamm.BaseModel(name="full model")
reduced_model = pybamm.BaseModel(name="reduced model")

models = [full_model, reduced_model]

R = pybamm.Parameter("Particle radius [m]")
D = pybamm.Parameter("Diffusion coefficient [m2.s-1]")
j = pybamm.Parameter("Interfacial current density [A.m-2]")
F = pybamm.Parameter("Faraday constant [C.mol-1]")
c0 = pybamm.Parameter("Initial concentration [mol.m-3]")

c = pybamm.Variable("Concentration [mol.m-3]", domain="negative particle")
c_av = pybamm.Variable("Average concentration [mol.m-3]")
# governing equations for full model
N = -D * pybamm.grad(c)  # flux
dcdt = -pybamm.div(N)
full_model.rhs = {c: dcdt}

# governing equations for reduced model
dc_avdt = -3 * j / R / F
reduced_model.rhs = {c_av: dc_avdt}

# initial conditions (these are the same for both models)
full_model.initial_conditions = {c: c0}
reduced_model.initial_conditions = {c_av: c0}

# boundary conditions (only required for full model)
lbc = pybamm.Scalar(0)
rbc = -j / F / D
full_model.boundary_conditions = {
    c: {"left": (lbc, "Neumann"), "right": (rbc, "Neumann")}
}
# full model
full_model.variables = {
    "Concentration [mol.m-3]": c,
    "Surface concentration [mol.m-3]": pybamm.surf(c),
    "Average concentration [mol.m-3]": pybamm.r_average(c),
}

# reduced model
reduced_model.variables = {
    "Concentration [mol.m-3]": pybamm.PrimaryBroadcast(c_av, "negative particle"),
    "Surface concentration [mol.m-3]": c_av,  # in this model the surface concentration is just equal to the scalar average concentration
    "Average concentration [mol.m-3]": c_av,
}
param = pybamm.ParameterValues(
    {
        "Particle radius [m]": 10e-6,
        "Diffusion coefficient [m2.s-1]": 3.9e-14,
        "Interfacial current density [A.m-2]": 1.4,
        "Faraday constant [C.mol-1]": 96485,
        "Initial concentration [mol.m-3]": 2.5e4,
    }
)
# geometry
r = pybamm.SpatialVariable(
    "r", domain=["negative particle"], coord_sys="spherical polar"
)
geometry = {"negative particle": {r: {"min": pybamm.Scalar(0), "max": R}}}
param.process_geometry(geometry)

# models
for model in models:
    param.process_model(model)
# mesh
submesh_types = {"negative particle": pybamm.Uniform1DSubMesh}
var_pts = {r: 20}
mesh = pybamm.Mesh(geometry, submesh_types, var_pts)

# discretisation
spatial_methods = {"negative particle": pybamm.FiniteVolume()}
disc = pybamm.Discretisation(mesh, spatial_methods)

# process models
for model in models:
    disc.process_model(model)

# loop over models to solve
t = np.linspace(0, 3600, 600)
solutions = [None] * len(models)  # create list to hold solutions
for i, model in enumerate(models):
    solver = pybamm.ScipySolver()
    solutions[i] = solver.solve(model, t)

# post-process the solution of the full model
c_full = solutions[0]["Concentration [mol.m-3]"]
c_av_full = solutions[0]["Average concentration [mol.m-3]"]


# post-process the solution of the reduced model
c_reduced = solutions[1]["Concentration [mol.m-3]"]
c_av_reduced = solutions[1]["Average concentration [mol.m-3]"]

# plot
r = mesh["negative particle"].nodes  # radial position


def plot(t):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 4))

    # Plot concetration as a function of r
    ax1.plot(r * 1e6, c_full(t=t, r=r), label="Full Model")
    ax1.plot(r * 1e6, c_reduced(t=t, r=r), label="Reduced Model")
    ax1.set_xlabel("Particle radius [microns]")
    ax1.set_ylabel("Concentration [mol.m-3]")
    ax1.legend()

    # Plot average concentration over time
    t_hour = np.linspace(0, 3600, 600)  # plot over full hour
    c_min = c_av_reduced(t=3600) * 0.98  # minimum axes limit
    c_max = param["Initial concentration [mol.m-3]"] * 1.02  # maximum axes limit

    ax2.plot(t_hour, c_av_full(t=t_hour), label="Full Model")
    ax2.plot(t_hour, c_av_reduced(t=t_hour), label="Reduced Model")
    ax2.plot([t, t], [c_min, c_max], "k--")  # plot line to track time
    ax2.set_xlabel("Time [s]")
    ax2.set_ylabel("Average concentration [mol.m-3]")
    ax2.legend()

    plt.tight_layout()
    plt.show()


import ipywidgets as widgets

widgets.interact(plot, t=widgets.FloatSlider(min=0, max=3600, step=1, value=0))

interactive(children=(FloatSlider(value=0.0, description='t', max=3600.0, step=1.0), Output()), _dom_classes=(…

<function __main__.plot(t)>

In [25]:
model = pybamm.lithium_ion.SPMe()
param = pybamm.ParameterValues("Chen2020")
fast_solver = pybamm.CasadiSolver(atol=1e-3, rtol=1e-3, mode="fast")    

experiment = pybamm.Experiment(
    [
        "Discharge at 1C until 2.3V",
        "Rest for 3 hours"
    ]
)

# 새로운 Simulation 객체 생성
new_sim = pybamm.Simulation(
    model,
    parameter_values=param,
    experiment=new_experiment,
    solver=fast_solver
)

# 새로운 실험을 현재 솔루션 상태에서 시작
continued_solution = new_sim.solve(initial_solution=initial_solution)



TypeError: BaseSolver.step() got an unexpected keyword argument 'initial_solution'

In [1]:

# Compares the full and lumped thermal models for a single layer Li-ion cell
#
import pybamm

pybamm.set_logging_level("INFO")

# load models
# for the full model we use the "x-full" thermal submodel, which means that we solve
# the thermal model in the x-direction for a single-layer pouch cell
# for the lumped model we use the "arbitrary" cell geometry, which means that we can
# specify the surface area for cooling and total heat transfer coefficient
full_thermal_model = pybamm.lithium_ion.SPMe(
    {"thermal": "x-full"}, name="full thermal model"
)
lumped_thermal_model = pybamm.lithium_ion.SPMe(
    {"cell geometry": "arbitrary", "thermal": "lumped"}, name="lumped thermal model"
)
models = [full_thermal_model, lumped_thermal_model]

# load parameter values, we will use the Marquis2019 parameter set
parameter_values = pybamm.ParameterValues("Marquis2019")
# for the "full" model we use a heat transfer coefficient of 5 W.m-2.K-1 on the large
# surfaces of the pouch and zero heat transfer coefficient on the tabs and edges
full_params = parameter_values.copy()
full_params.update(
    {
        "Negative current collector surface heat transfer coefficient [W.m-2.K-1]"
        "": 5,
        "Positive current collector surface heat transfer coefficient [W.m-2.K-1]"
        "": 5,
        "Negative tab heat transfer coefficient [W.m-2.K-1]": 0,
        "Positive tab heat transfer coefficient [W.m-2.K-1]": 0,
        "Edge heat transfer coefficient [W.m-2.K-1]": 0,
    }
)
# for the lumped model we set the "Total heat transfer coefficient [W.m-2.K-1]"
# parameter as well as the "Cell cooling surface area [m2]" and "Cell volume [m3]
# parameters. Since the "full" model only accounts for cooling from the large surfaces
# of the pouch, we set the "Surface area for cooling [m2]" parameter to the area of the
# large surfaces of the pouch, and the total heat transfer coefficient to 5 W.m-2.K-1
A = parameter_values["Electrode width [m]"] * parameter_values["Electrode height [m]"]
contributing_layers = [
    "Negative current collector",
    "Negative electrode",
    "Separator",
    "Positive electrode",
    "Positive current collector",
]
total_thickness = sum(
    [parameter_values[layer + " thickness [m]"] for layer in contributing_layers]
)
electrode_volume = (
    total_thickness
    * parameter_values["Electrode height [m]"]
    * parameter_values["Electrode width [m]"]
)
lumped_params = parameter_values.copy()
lumped_params.update(
    {
        "Total heat transfer coefficient [W.m-2.K-1]": 5,
        "Cell cooling surface area [m2]": 2 * A,
        "Cell volume [m3]": electrode_volume,
    }
)

# loop over the models and solve
params = [full_params, lumped_params]
sols = []
for model, param in zip(models, params):
    sim = pybamm.Simulation(model, parameter_values=param)
    sim.solve([0, 3600])
    sols.append(sim.solution)

# plot
output_variables = [
    "Voltage [V]",
    "X-averaged cell temperature [K]",
    "Cell temperature [K]",
]
pybamm.dynamic_plot(sols, output_variables)

2024-04-04 11:55:06.432 - [INFO] base_model._build_model(652): Start building full thermal model
2024-04-04 11:55:06.590 - [INFO] base_battery_model.build_model(1073): Finish building full thermal model
2024-04-04 11:55:06.611 - [INFO] base_model._build_model(652): Start building lumped thermal model
2024-04-04 11:55:06.695 - [INFO] base_battery_model.build_model(1073): Finish building lumped thermal model
2024-04-04 11:55:06.710 - [INFO] parameter_values.process_model(394): Start setting parameters for full thermal model
2024-04-04 11:55:06.924 - [INFO] parameter_values.process_model(463): Finish setting parameters for full thermal model
2024-04-04 11:55:06.927 - [INFO] discretisation.process_model(145): Start discretising full thermal model
2024-04-04 11:55:06.945 - [INFO] discretisation.remove_independent_variables_from_rhs(1126): removing variable Discharge capacity [A.h] from rhs
2024-04-04 11:55:06.948 - [INFO] discretisation.remove_independent_variables_from_rhs(1126): removing 

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

In [21]:
import pybamm

# Define the model
model = pybamm.lithium_ion.SPMe()

# 초기 배터리 정보
param = pybamm.ParameterValues("Chen2020")

#  방전 후 사용
experiment = pybamm.Experiment(
    [
        "Discharge at 1C until 2.3V",
        "Rest for 3 hours"
    ]
)

# Solver settings
fast_solver = pybamm.CasadiSolver(atol=1e-3, rtol=1e-3, mode="fast")

# Create simulation
fast_sim = pybamm.Simulation(model, parameter_values=param, experiment=experiment, solver=fast_solver)

# Define action function
def action(current, delta_t):
    global start_hour
    end_hour = start_hour + delta_t
    pybamm.step.string(f"Charge at {current}C for {delta_t} min")
    solution = fast_sim.solve([start_hour, end_hour])
    start_hour = end_hour
    voltage = solution["Terminal voltage [V]"].entries[:]  # Get terminal voltage
    time = solution["Terminal voltage [V]"].entries[:]  # Get terminal voltage
    print("Terminal Voltage:", voltage)
    

# Perform actions
start_hour = 0
for i in range(6):
    action(1, 10)


2024-04-02 22:14:39.793 - [WARNING] simulation.solve(590): Ignoring t_eval as solution times are specified by the experiment
2024-04-02 22:14:40.293 - [WARNING] simulation.solve(590): Ignoring t_eval as solution times are specified by the experiment
2024-04-02 22:14:40.455 - [WARNING] simulation.solve(590): Ignoring t_eval as solution times are specified by the experiment


Terminal Voltage: [4.03635574 3.94216222 3.93096432 3.92355714 3.91133262 3.89510077
 3.87711307 3.85894122 3.84177565 3.82576744 3.81125519 3.79786192
 3.78538264 3.77344775 3.76173239 3.7497607  3.73687999 3.72204262
 3.70403072 3.68259638 3.65965868 3.63805896 3.6187448  3.60126226
 3.58517583 3.57034366 3.55665486 3.54403032 3.53241578 3.5216871
 3.51156449 3.50184804 3.49240887 3.48309133 3.47368349 3.4639888
 3.45378726 3.44280336 3.43075357 3.41733811 3.40234928 3.38580566
 3.36806441 3.34979861 3.33181418 3.31476556 3.29889657 3.28405577
 3.26961281 3.25432277 3.23590972 3.21050096 3.17307158 3.12166844
 3.06259466 3.00391883 2.94329241 2.86712943 2.7533497  2.56740426
 2.30001003 2.47800355 2.8010386  2.84090324 2.85458706 2.86124065
 2.86554159 2.86835485 2.87022887 2.87159296 2.87267996 2.87354434
 2.87436354 2.87504485 2.87556917 2.87605663 2.8764521  2.87675584
 2.8770358  2.87725782 2.8774257  2.87758093 2.87770689 2.87780657
 2.87788297 2.87793906 2.87798207 2.87801417 2

2024-04-02 22:14:40.635 - [WARNING] simulation.solve(590): Ignoring t_eval as solution times are specified by the experiment
2024-04-02 22:14:40.837 - [WARNING] simulation.solve(590): Ignoring t_eval as solution times are specified by the experiment


Terminal Voltage: [4.03635574 3.94216222 3.93096432 3.92355714 3.91133262 3.89510077
 3.87711307 3.85894122 3.84177565 3.82576744 3.81125519 3.79786192
 3.78538264 3.77344775 3.76173239 3.7497607  3.73687999 3.72204262
 3.70403072 3.68259638 3.65965868 3.63805896 3.6187448  3.60126226
 3.58517583 3.57034366 3.55665486 3.54403032 3.53241578 3.5216871
 3.51156449 3.50184804 3.49240887 3.48309133 3.47368349 3.4639888
 3.45378726 3.44280336 3.43075357 3.41733811 3.40234928 3.38580566
 3.36806441 3.34979861 3.33181418 3.31476556 3.29889657 3.28405577
 3.26961281 3.25432277 3.23590972 3.21050096 3.17307158 3.12166844
 3.06259466 3.00391883 2.94329241 2.86712943 2.7533497  2.56740426
 2.30001003 2.47800355 2.8010386  2.84090324 2.85458706 2.86124065
 2.86554159 2.86835485 2.87022887 2.87159296 2.87267996 2.87354434
 2.87436354 2.87504485 2.87556917 2.87605663 2.8764521  2.87675584
 2.8770358  2.87725782 2.8774257  2.87758093 2.87770689 2.87780657
 2.87788297 2.87793906 2.87798207 2.87801417 2

2024-04-02 22:14:41.015 - [WARNING] simulation.solve(590): Ignoring t_eval as solution times are specified by the experiment


Terminal Voltage: [4.03635574 3.94216222 3.93096432 3.92355714 3.91133262 3.89510077
 3.87711307 3.85894122 3.84177565 3.82576744 3.81125519 3.79786192
 3.78538264 3.77344775 3.76173239 3.7497607  3.73687999 3.72204262
 3.70403072 3.68259638 3.65965868 3.63805896 3.6187448  3.60126226
 3.58517583 3.57034366 3.55665486 3.54403032 3.53241578 3.5216871
 3.51156449 3.50184804 3.49240887 3.48309133 3.47368349 3.4639888
 3.45378726 3.44280336 3.43075357 3.41733811 3.40234928 3.38580566
 3.36806441 3.34979861 3.33181418 3.31476556 3.29889657 3.28405577
 3.26961281 3.25432277 3.23590972 3.21050096 3.17307158 3.12166844
 3.06259466 3.00391883 2.94329241 2.86712943 2.7533497  2.56740426
 2.30001003 2.47800355 2.8010386  2.84090324 2.85458706 2.86124065
 2.86554159 2.86835485 2.87022887 2.87159296 2.87267996 2.87354434
 2.87436354 2.87504485 2.87556917 2.87605663 2.8764521  2.87675584
 2.8770358  2.87725782 2.8774257  2.87758093 2.87770689 2.87780657
 2.87788297 2.87793906 2.87798207 2.87801417 2

In [81]:
#
# Compares the full and lumped thermal models for a single layer Li-ion cell
#
import pybamm

pybamm.set_logging_level("INFO")

# load models
# for the full model we use the "x-full" thermal submodel, which means that we solve
# the thermal model in the x-direction for a single-layer pouch cell
# for the lumped model we use the "arbitrary" cell geometry, which means that we can
# specify the surface area for cooling and total heat transfer coefficient
full_thermal_model = pybamm.lithium_ion.SPMe(
    {"thermal": "x-full"}, name="full thermal model"
)
lumped_thermal_model = pybamm.lithium_ion.SPMe(
    {"cell geometry": "arbitrary", "thermal": "lumped"}, name="lumped thermal model"
)
models = [full_thermal_model, lumped_thermal_model]

# load parameter values, we will use the Marquis2019 parameter set
parameter_values = pybamm.ParameterValues("Marquis2019")
# for the "full" model we use a heat transfer coefficient of 5 W.m-2.K-1 on the large
# surfaces of the pouch and zero heat transfer coefficient on the tabs and edges
full_params = parameter_values.copy()
full_params.update(
    {
        "Negative current collector surface heat transfer coefficient [W.m-2.K-1]"
        "": 5,
        "Positive current collector surface heat transfer coefficient [W.m-2.K-1]"
        "": 5,
        "Negative tab heat transfer coefficient [W.m-2.K-1]": 0,
        "Positive tab heat transfer coefficient [W.m-2.K-1]": 0,
        "Edge heat transfer coefficient [W.m-2.K-1]": 0,
    }
)
# for the lumped model we set the "Total heat transfer coefficient [W.m-2.K-1]"
# parameter as well as the "Cell cooling surface area [m2]" and "Cell volume [m3]
# parameters. Since the "full" model only accounts for cooling from the large surfaces
# of the pouch, we set the "Surface area for cooling [m2]" parameter to the area of the
# large surfaces of the pouch, and the total heat transfer coefficient to 5 W.m-2.K-1
A = parameter_values["Electrode width [m]"] * parameter_values["Electrode height [m]"]
contributing_layers = [
    "Negative current collector",
    "Negative electrode",
    "Separator",
    "Positive electrode",
    "Positive current collector",
]
total_thickness = sum(
    [parameter_values[layer + " thickness [m]"] for layer in contributing_layers]
)
electrode_volume = (
    total_thickness
    * parameter_values["Electrode height [m]"]
    * parameter_values["Electrode width [m]"]
)
lumped_params = parameter_values.copy()
lumped_params.update(
    {
        "Total heat transfer coefficient [W.m-2.K-1]": 5,
        "Cell cooling surface area [m2]": 2 * A,
        "Cell volume [m3]": electrode_volume,
    }
)

# loop over the models and solve
params = [full_params, lumped_params]
sols = []
for model, param in zip(models, params):
    sim = pybamm.Simulation(model, parameter_values=param)
    sim.solve([0, 3600])
    sols.append(sim.solution)

# plot
output_variables = [
    "Voltage [V]",
    "X-averaged cell temperature [K]",
    "Cell temperature [K]",
]
pybamm.dynamic_plot(sols, output_variables)

2024-04-02 19:41:23.051 - [INFO] base_model._build_model(652): Start building full thermal model
2024-04-02 19:41:23.190 - [INFO] base_battery_model.build_model(1073): Finish building full thermal model
2024-04-02 19:41:23.221 - [INFO] base_model._build_model(652): Start building lumped thermal model
2024-04-02 19:41:23.352 - [INFO] base_battery_model.build_model(1073): Finish building lumped thermal model
2024-04-02 19:41:23.375 - [INFO] parameter_values.process_model(394): Start setting parameters for full thermal model
2024-04-02 19:41:23.718 - [INFO] parameter_values.process_model(463): Finish setting parameters for full thermal model
2024-04-02 19:41:23.718 - [INFO] discretisation.process_model(145): Start discretising full thermal model
2024-04-02 19:41:23.767 - [INFO] discretisation.remove_independent_variables_from_rhs(1126): removing variable Discharge capacity [A.h] from rhs
2024-04-02 19:41:23.767 - [INFO] discretisation.remove_independent_variables_from_rhs(1126): removing 

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

In [82]:
#
# Constant-current constant-voltage charge with US06 Drive Cycle using Experiment Class.
#
import pybamm
import pandas as pd
import os

os.chdir(pybamm.__path__[0] + "/..")

pybamm.set_logging_level("INFO")

# import drive cycle from file
drive_cycle_current = pd.read_csv(
    "pybamm/input/drive_cycles/US06.csv", comment="#", header=None
).to_numpy()


# Map Drive Cycle
def map_drive_cycle(x, min_op_value, max_op_value):
    min_ip_value = x[:, 1].min()
    max_ip_value = x[:, 1].max()
    x[:, 1] = (x[:, 1] - min_ip_value) / (max_ip_value - min_ip_value) * (
        max_op_value - min_op_value
    ) + min_op_value
    return x


# Map current drive cycle to voltage and power
drive_cycle_power = map_drive_cycle(drive_cycle_current, 1.5, 3.5)

experiment = pybamm.Experiment(
    [
        (
            "Charge at 1 A until 4.0 V",
            "Hold at 4.0 V until 50 mA",
            "Rest for 30 minutes",
            pybamm.sSolutiontep.current(drive_cycle_current),
            "Rest for 30 minutes",
            pybamm.step.power(drive_cycle_power),
            "Rest for 30 minutes",
        ),
    ],
)

model = pybamm.lithium_ion.DFN()
sim = pybamm.Simulation(model, experiment=experiment, solver=pybamm.CasadiSolver())
sim.solve()

# Show all plots
sim.plot()

2024-04-02 19:43:26.809 - [INFO] base_model._build_model(652): Start building Doyle-Fuller-Newman model
2024-04-02 19:43:26.863 - [INFO] base_battery_model.build_model(1073): Finish building Doyle-Fuller-Newman model
2024-04-02 19:43:26.866 - [INFO] callbacks.on_experiment_start(164): Start running experiment
2024-04-02 19:43:26.868 - [INFO] parameter_values.process_model(394): Start setting parameters for Doyle-Fuller-Newman model
2024-04-02 19:43:26.974 - [INFO] parameter_values.process_model(463): Finish setting parameters for Doyle-Fuller-Newman model
2024-04-02 19:43:26.974 - [INFO] parameter_values.process_model(394): Start setting parameters for Doyle-Fuller-Newman model
2024-04-02 19:43:27.094 - [INFO] parameter_values.process_model(463): Finish setting parameters for Doyle-Fuller-Newman model
2024-04-02 19:43:27.094 - [INFO] parameter_values.process_model(394): Start setting parameters for Doyle-Fuller-Newman model
2024-04-02 19:43:27.208 - [INFO] parameter_values.process_mode

interactive(children=(FloatSlider(value=0.0, description='t', max=2.13580901471914, step=0.0213580901471914), …